# Markov-Shannon, a simple text generator
Nowadays generating texts using Transformers neural network architectures reached high level of sophistication.  But the idea to generate text from existing examples is very old.  
For example, a **Markov test generator** (the idea for this is apparently due to Claude Shannon who described it in his Communication Theory paper) is named after Andrey Markov (1856-1922), who probably never saw one (at the word level, at least). A Markov chain is essentially a finite-state automaton where all the transitions are assigned probabilities (I described them previously in the POS tagging series part).  
It works very simply:  given a corpus of text, each word in the corpus becomes a state and each two-word sequence is a transition to another state. It’s assigned a probability determined by how many times it appears.   
In his paper *A Mathematical Theory of Communication*, Shannon calls completely random series zero-ordered approximations. Putting randomly selected words after each other yields totally unintelligible lines. As one selects words according to their frequency in a huge corpus, the resulting text gets more natural. If we go further, and we take two-word or three-word or n-word sequences, we get better and better results. One can see these n-word sequences (or n-grams) as transitions from one word to the other.  
  
This is an example of a text generator using this simple idea.

We start by loading some text dataset. Since a Markov generator becomes interesting only with a pretty large corpus, we use a couple of Shakespeare's works.  
## Read the data

In [1]:
import os

In [2]:
dirname = '../datasets/shakespeare/'
lines = [] # storing all the lines in a variable. 
for filename in os.listdir(dirname):
    with open(os.path.join(dirname, filename)) as files:
        for line in files:
            # remove leading and trailing whitespace
            pure_line = line.strip()
            
            # if pure_line is not the empty string,
            if pure_line:
                # append it to the list
                lines.append(pure_line)
                
n_lines = len(lines)

print(f"Number of lines: {n_lines}")
print(f"Sample line at position 0 {lines[0]}")
print(f"Sample line at position 999 {lines[999]}")    

Number of lines: 20907
Sample line at position 0 THE TEMPEST
Sample line at position 999 Who shall be of as little memory


In [3]:
print("\n".join(lines[506:514]))

CALIBAN	As wicked dew as e'er my mother brush'd
With raven's feather from unwholesome fen
Drop on you both! a south-west blow on ye
And blister you all o'er!
PROSPERO	For this, be sure, to-night thou shalt have cramps,
Side-stitches that shall pen thy breath up; urchins
Shall, for that vast of night that they may work,
All exercise on thee; thou shalt be pinch'd


A total of 20K lines have been read and they form the input corpus.  
### Clean the data
For simplicity, we are going to clean them from some punctuation and symbols and also lowercase everything.


In [4]:
# go through each line
for i, line in enumerate(lines):
    # convert to all lowercase
    lines[i] = line.lower()

print(f"Number of lines: {n_lines}")
print(f"Sample line at position 0 {lines[0]}")
print(f"Sample line at position 999 {lines[999]}")

Number of lines: 20907
Sample line at position 0 the tempest
Sample line at position 999 who shall be of as little memory


In [5]:
def clean(hdln):
    hdln = hdln.replace("'", "")
    hdln = hdln.replace('"', "")
    hdln = hdln.replace("…", "")
    hdln = hdln.replace("”", "")
    hdln = hdln.replace("`", "")
    hdln = hdln.replace("’", "")
    hdln = hdln.replace("“", "")
    hdln = hdln.replace("(", "")
    hdln = hdln.replace(")", "")
    hdln = hdln.replace("#", "")
    return hdln


In [6]:
cleanlines = [e.strip() for e in lines]
cleanlines = [e.split("|")[0] for e in cleanlines]
cleanlines = [clean(e) for e in cleanlines]
print("\n".join(cleanlines[506:514]))

caliban	as wicked dew as eer my mother brushd
with ravens feather from unwholesome fen
drop on you both! a south-west blow on ye
and blister you all oer!
prospero	for this, be sure, to-night thou shalt have cramps,
side-stitches that shall pen thy breath up; urchins
shall, for that vast of night that they may work,
all exercise on thee; thou shalt be pinchd


In [7]:
corpus = "\n".join(cleanlines)


## Tokenize
Next we will extract all tokens from the corpus and we use the old good NLTK tokenizer.  


In [8]:
from nltk.tokenize import word_tokenize

In [9]:
tokens = []
line_length = []

for line in cleanlines:
    tkns = word_tokenize(line)
    line_length.append(len(tkns))
    tokens.extend(tkns)


In [10]:
print(f"The average line length is {sum(line_length)/len(line_length)}")


The average line length is 8.942268139857465


## Get token frequency
To improve the word generation we can use weights based on the token's frequency when randomly picking a word

In [11]:
from collections import Counter
# a sub-class that is used to count hashable objects. 

In [12]:
# count frequencies
token_freq = Counter(tokens)
type(token_freq)

collections.Counter

In [13]:
token_freq.most_common(5)

[(',', 14890), ('.', 6195), ('the', 4843), ('and', 4499), ('i', 3520)]

A Counter object has key and values: the word itself and its frequency:

In [14]:
print(list(token_freq.keys())[:5]) # the first five key,values in the corpus
print(list(token_freq.values())[:5])

['the', 'tempest', 'dramatis', 'personae', 'alonso']
[4843, 20, 6, 6, 50]


In [15]:
list(token_freq.items())[:5][:5]  # key and values

[('the', 4843),
 ('tempest', 20),
 ('dramatis', 6),
 ('personae', 6),
 ('alonso', 50)]

## First order text generation
Having generated and saved the frequency tables, we have a simple model and we can start working on text generation. Our simple generator takes as arguments the corpus, an initial seed and an order n (how many n-grams to consider).   
The first order considers only the tokens and their frequency in the corpus. Each generated token is randomly selected. Also notes that the seed has no impact on the next tokens.  
Don't expect good results from this.

In [16]:
import random

In [17]:
initialSeed ="max"

In [18]:
genLine = [initialSeed]
for i in range(10): # we generate ten tokens randomly but frequency-based   
    w = random.choices(population=list(token_freq.keys()),
                                  weights=list(token_freq.values()))[0]
    genLine.append(w)
    
genLine

['max',
 'the',
 'to',
 'reeking',
 'as',
 'day',
 'gave',
 'it',
 'you',
 'our',
 'same']

In [19]:
print(' '.join([str(item) for item in genLine])) # this is the result, formatted

max the to reeking as day gave it you our same


This generator is quite poor because the words are not considered together, just random (although based on the frequency, this is why filler and stop words come out often).  
## Second-order text generation, using bigrams

For better results, we can consider bigrams, so two tokens together.  
For this we first need to get all possible bigrams and their frequency.

In [20]:
from nltk import ngrams

In [21]:
bigrams = []

for line in cleanlines:
    tkns = word_tokenize(line)
    bgrms = list(ngrams(tkns, 2))
    bigrams.extend(bgrms)


In [22]:
type(bigrams)

list

In [23]:
bigram_freq = Counter(bigrams)


In [24]:
bigram_total = sum(bigram_freq.values())
bigram_total

166090

In [25]:
bigram_freq.most_common(5)

[((',', 'and'), 971),
 ((',', 'i'), 492),
 ((',', 'my'), 478),
 (('my', 'lord'), 436),
 (('[', 'enter'), 328)]

Note that each key is now a tuple formed by two tokens

In [26]:
print(list(bigram_freq.keys())[:5])
print(list(bigram_freq.values())[:5])

[('the', 'tempest'), ('dramatis', 'personae'), ('alonso', 'king'), ('king', 'of'), ('of', 'naples')]
[14, 6, 1, 20, 11]


In [27]:
list(bigram_freq.items())[:5][:5]  # key and values

[(('the', 'tempest'), 14),
 (('dramatis', 'personae'), 6),
 (('alonso', 'king'), 1),
 (('king', 'of'), 20),
 (('of', 'naples'), 11)]

The principle of second order generation is slightly different, as we can use the seed to find bigrams with it.  
If the seed is not in the corpus, it randomly selects a word, otherwise it returns a weighted choice from the  tokens.

In [28]:
def token_generator_2nd_order(seed, DEBUG=False):
    
    if seed not in tokens:
        seed = random.choice(list(token_freq.keys())) # get another word
        if DEBUG:
            print("New seed is: ", seed)
        
    bgms = {k: v for (k, v) in bigram_freq.items() if k[0] == seed}
    if DEBUG:
        print("Bigrams are: ", bgms)
        
    wds = [e[1] for e in bgms.keys()]
        
    if wds:
        weights = [float(e) for e in bgms.values()]
        return random.choices(population=wds, weights=weights)[0]
    else:
        return random.choices(population=list(token_freq.keys()),
                                  weights=list(token_freq.values()))[0]


In [29]:
genLine = []
genLine.append(initialSeed)
for i in range(1,10): # we generate ten tokens   
    w = token_generator_2nd_order(genLine[-1]) # pass the last generated word as new seed
    genLine.append(w)
    
genLine

['max', 'did', 'fight', 'for', 'it', ',', 'scarce', ',', 'right', 'use']

In [30]:
print(' '.join([str(item) for item in genLine])) # this is the result, formatted

max did fight for it , scarce , right use


It looks slightly better, I like how the sentence started!  
We can further improve by looking at 3 or 4 tokens together.  

## N-th order text generation using 3 and 4-grams
Again, we first generate the 3 and 4-grams collections, with their frequency:

In [31]:
trigrams = []
fourgrams = []

for line in cleanlines:
    tkns = word_tokenize(line)
    trgms = list(ngrams(tkns, 3))
    trigrams.extend(trgms)
    frgrms = list(ngrams(tkns, 4))
    fourgrams.extend(frgrms)


In [32]:
trigram_freq = Counter(trigrams)
fourgram_freq = Counter(fourgrams)


In [33]:
trigram_total = sum(trigram_freq.values())
fourgram_total = sum(fourgram_freq.values())
trigram_total

145358

In [34]:
fourgram_total

124858

In [35]:
trigram_freq.most_common(5)

[((',', 'my', 'lord'), 271),
 (('king', 'richard', 'iii'), 184),
 (('my', 'lord', ','), 175),
 ((',', 'sir', ','), 119),
 (('[', 'exeunt', ']'), 96)]

In [36]:
list(trigram_freq.items())[:5][:5]  # key and values

[(('alonso', 'king', 'of'), 1),
 (('king', 'of', 'naples'), 5),
 (('of', 'naples', '.'), 4),
 (('sebastian', 'his', 'brother'), 1),
 (('his', 'brother', '.'), 3)]

In [37]:
def token_generator_nth_order(order, seed, seed2, seed3=None, DEBUG=False):
    
    if seed not in tokens:
        seed = random.choice(list(token_freq.keys()))
        if DEBUG:
            print("New seed is: ", seed)
        
    if order == 3:
                        # we need two words to find the third
        tgms = {k: v for (k, v) in trigram_freq.items() if k[:2] == (seed,seed2)}
        if DEBUG:
            print("Trigrams are: ", tgms)
            
        wds = [e[2] for e in tgms.keys()]
        if wds:
            weights = [float(e) for e in tgms.values()]
            return random.choices(population=wds, weights=weights)[0]
        else:
            return random.choices(population=list(token_freq.keys()),
                                  weights=list(token_freq.values()))[0]
        
    else:
                        # we need three words to find the fourth
        ngms = {k: v for (k, v) in fourgram_freq.items() if k[:3] == (seed,seed2,seed3)}
        if DEBUG:
            print("Fourgrams are: ", ngms)
            
        wds = [e[3] for e in ngms.keys()]
        if wds:
            weights = [float(e) for e in ngms.values()]
            return random.choices(population=wds, weights=weights)[0]
        else:
            return random.choices(population=list(token_freq.keys()),
                                  weights=list(token_freq.values()))[0]

In [39]:
# 3rd order text generation

genLine = []
genLine.append(initialSeed)
w2 = token_generator_2nd_order(genLine[-1]) # we need a second word
genLine.append(w2)

for i in range(1,10): # we generate ten tokens   
    w = token_generator_nth_order(3, genLine[-2], genLine[-1]) # pass the last two generated words as new seeds
    genLine.append(w)
    
genLine

['max',
 'of',
 'place',
 'and',
 'duty',
 'that',
 'i',
 'respect',
 ',',
 'for',
 'joy']

In [40]:
print(' '.join([str(item) for item in genLine])) # this is the result, formatted

max of place and duty that i respect , for joy


In [43]:
# 4th order text generation

genLine = []
genLine.append(initialSeed)
w2 = token_generator_2nd_order(initialSeed) # now we need a second and third word
genLine.append(w2)
w3 = token_generator_nth_order(3, initialSeed, w2) 
genLine.append(w3)

for i in range(1,10): # we generate ten tokens   
    w = token_generator_nth_order(4, genLine[-3], genLine[-2], genLine[-1]) # pass the last two generated words as new seeds
    genLine.append(w)
    
genLine

['max',
 'he',
 ',',
 ',',
 'of',
 'the',
 'first',
 'and',
 'second',
 'cause',
 ':',
 'i']

In [44]:
print(' '.join([str(item) for item in genLine])) # this is the result, formatted

max he , , of the first and second cause : i


## Conclusion
Let's put all together in a class. It can be initialised using lines taken from any text, after which will have tokens and ngrams. One method is to generate the next token and then several methods to get sentences.  
I introduce also another change: if there is no suitable fourgrams, then it will pick a random word independently from the weights. It adds a bit of unpredictability to the sentence and avoid too many articles and other stop words. It's kind of tuning the temperature of the text generation.

In [45]:
class ShannonMarkovTokenGenerator:
    def __init__(self, l):
        print("Generating the model from total lines in input: ", len(l))
        lines = [e.split("|")[0] for e in l]
            # go through each line and clean it
        for i, line in enumerate(lines):
            # convert to all lowercase
            lines[i] = line.lower()
            lines[i] = line.strip()
            lines[i] = line.replace("'", "")
            lines[i] = line.replace('"', "")
            lines[i] = line.replace("…", "")
            lines[i] = line.replace("”", "")
            lines[i] = line.replace("`", "")
            lines[i] = line.replace("“", "")
            lines[i] = line.replace("(", "")
            lines[i] = line.replace("(", "")
            lines[i] = line.replace("#", "")

            # prepare tokens and ngrams
        tokens = []
        bigrams = []
        trigrams = []
        fourgrams = []
        line_length = []

        for line in lines:
            tkns = word_tokenize(line)
            bgrms = list(ngrams(tkns, 2))
            trgms = list(ngrams(tkns, 3))
            frgrms = list(ngrams(tkns, 4))
            
            tokens.extend(tkns)
            bigrams.extend(bgrms)
            trigrams.extend(trgms)
            fourgrams.extend(frgrms)

            line_length.append(len(tkns))

            # tokens and ngrams are stored
        self.corpusTokens = tokens
        self.tokensFrequence = Counter(tokens)
        self.bigramsFrequence = Counter(bigrams)
        self.trigramsFrequence = Counter(trigrams)
        self.fourgramsFrequence = Counter(fourgrams)

            # let's store also a couple of stats about the text
        self.tokensTotal = sum(self.tokensFrequence.values())
        self.avgLineLength = sum(line_length)/len(line_length)
        self.lineLength = round(self.avgLineLength) + 1

        print("Done. Total tokens extracted: ", self.tokensTotal)
    
    
    def token_generator(self, seed, order, seed2=None, seed3=None):
        '''
        generate a word (token) based on an input n-gram and the text stored
        order = identifies if bigram, trigram, etc.; number between 1 and 4
        seed = first input token
        seed2 = optional. second input token
        seed3 = optional. third input token
        returns a token (string of characters)
        '''
    
            # check that the first seed is in the corpus, otherwise pick randomly a new one
            # we assume that the other seeds are coming from the corpus, as they have been previously generated
        if seed not in tokens:
            seed = random.choice(list(self.tokensFrequence.keys()))
            
        if (not seed2) & (order > 2):
            seed2 = token_generator_2nd_order(seed) # needed only first time
  
            # Second-order
            # select a new token based on the given seed and the built bigrams
        if order == 2:
            bgms = {k: v for (k, v) in self.bigramsFrequence.items() if k[0] == seed}

            wds = [e[1] for e in bgms.keys()]  # list of all possible words
                    # if no available words from the bigrams, pick a random one from corpus
            if wds:
                weights = [float(e) for e in bgms.values()]
                return random.choices(population=wds, weights=weights)[0]
            else:
                return random.choices(population=list(self.tokensFrequence.keys()),
                                  weights=list(self.tokensFrequence.values()))[0]
            # Third-order
            # select a new token based on the given two seeds and the built trigrams
        elif order == 3:

            tgms = {k: v for (k, v) in trigram_freq.items() if k[:2] == (seed,seed2)}
            wds = [e[2] for e in tgms.keys()]
            if wds:
                weights = [float(e) for e in tgms.values()]
                return random.choices(population=wds, weights=weights)[0]
            else:
                return random.choice(list(self.tokensFrequence.keys()))
        
        else:
            ngms = {k: v for (k, v) in fourgram_freq.items() if k[:3] == (seed,seed2,seed3)}
            wds = [e[3] for e in ngms.keys()]
            if wds:
                weights = [float(e) for e in ngms.values()]
                return random.choices(population=wds, weights=weights)[0]
            else:
                return random.choice(list(self.tokensFrequence.keys()))  # no more weights!



    def generate1stOrderLine(self, nLines=1):
        '''
        generate one or more line of text (string) based on the text stored
        nLines = optional. How many lines to generate. Default = 1
        returns a string: nLines (separated by newline) of averageLineLength words
        '''
        
        genLines = ""
        for l in range(nLines):
            genLine = []
            for i in range(self.lineLength):    
                w = random.choices(population=list(self.tokensFrequence.keys()),
                                  weights=list(self.tokensFrequence.values()))[0]
                genLine.append(w)
        
            strLine = ' '.join([str(item) for item in genLine]) # make the list of words a string
            genLines += strLine 
            genLines += "\n"
        
        return genLines


    def generate2ndOrderLine(self, seed, nLines=1):            
        '''
        generate one or more line of text (string) based on an initial seed and the text stored, using bigrams
        seed = first input token
        nLines = optional. How many lines to generate. Default = 1
        returns a string: nLines (separated by newline) of averageLineLength words
        '''
        if seed not in tokens:
            seed = random.choice(list(self.tokensFrequence.keys()))
        w2 = self.token_generator(seed, 2) # first word from seed
                    
        genLine = []
        genLine.append(seed)    # start the line with the seed and the first word            
        genLine.append(w2)
        genLines = seed + " " + w2 + " "

        for l in range(nLines):
            
            for i in range(1, self.lineLength):  

                w = self.token_generator(genLine[-2], 2, genLine[-1])

                genLine.append(w)
        
            strLine = ' '.join([str(item) for item in genLine[2:]])
            genLines += strLine 
            genLines += "\n"
            
            # we keep only the last two words for next line
            genLine = genLine[-2:]
        
        return genLines

    def generate3rdOrderLine(self, seed, nLines=1):            
        '''
        generate one or more line of text (string) based on an initial seed and the text stored, using trigrams
        seed = first input token
        nLines = optional. How many lines to generate. Default = 1
        returns a string: nLines (separated by newline) of averageLineLength words
        '''
        if seed not in tokens:
            seed = random.choice(list(self.tokensFrequence.keys()))
        w2 = self.token_generator(seed, 2) # first word from seed
        w3 = self.token_generator(seed, 3, w2) # second word from the two above
                    
        genLine = []
        genLine.append(seed)        
        genLine.append(w2) 
        genLine.append(w3)

        genLines = seed + " " + w2 + " " + w3 + " "

        for l in range(nLines):
            
            for i in range(1, self.lineLength):  
                w = self.token_generator(genLine[-3], 3, genLine[-2], genLine[-1])

                genLine.append(w)
        
            strLine = ' '.join([str(item) for item in genLine[3:]])
            genLines += strLine 
            genLines += "\n"

                        # we keep only the last three words for next line
            genLine = genLine[-3:]

        return genLines
    
    
    def generate4thOrderLine(self, seed, nLines=1):       
        '''
        generate one or more line of text (string) based on an initial seed and the text stored, using fourgrams
        seed = first input token
        nLines = optional. How many lines to generate. Default = 1
        returns a string: nLines (separated by newline) of averageLineLength words
        '''
            
        if seed not in tokens:
            seed = random.choice(list(self.tokensFrequence.keys()))
        w2 = self.token_generator(seed, 2) # first word
        w3 = self.token_generator(seed, 3, w2) # second word 
                    
        genLine = []
        genLine.append(seed)        
        genLine.append(w2) 
        genLine.append(w3)

        genLines = seed + " " + w2 + " " + w3 + " "

        for l in range(nLines):
            
            for i in range(1, self.lineLength):  
                w = self.token_generator(genLine[-3], 4, genLine[-2], genLine[-1])

                genLine.append(w)
        
            strLine = ' '.join([str(item) for item in genLine[3:]])
            genLines += strLine 
            genLines += "\n"

                        # we keep only the last two words for next line
            genLine = genLine[-3:]

        return genLines

In [46]:
model = ShannonMarkovTokenGenerator(lines)

Generating the model from total lines in input:  20907
Done. Total tokens extracted:  190812


In [47]:
print(f"The average line length is {model.avgLineLength}")


The average line length is 9.1267039747453


You can access directly the tokens and their frequency:

In [48]:
model.tokensFrequence.most_common(5)

[(',', 14890), ('.', 6196), ('the', 4836), ('and', 4499), ('i', 3871)]

And now we generate the different orders lines, two lines each time:

In [49]:
print(model.generate1stOrderLine(2))

how devise swear bless too been is traitors yet themselves
, sight work lord . fertile forth for had in



In [50]:
print(model.generate2ndOrderLine("how", 2))

how is't thou with hast me enchanted my her greatness ,
will is receive her them sorrow wear and thou



In [51]:
print(model.generate3rdOrderLine("how", 2))

how is it it out accommodation . weigh'st begone indignation sparingly seest
re-deliver wheat pain winters twenty skyish wave-worn incharitable thanes



In [52]:
print(model.generate4thOrderLine("how", 2))

how satisfied , my lord , what is your tidings ? commencing
whiff daylight stripping entertainer norways unswear antres brakenbury fulsome



Note that now the 3rd and 4th order get more random words and less articles.  
That's all. Feel free to try with different corpus of text.